In [1]:
from datasets import load_dataset

# load IMDb dataset and take a small sample
dataset = load_dataset("imdb", split="train[:1%]")
dataset[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [2]:
def preprocess(batch):
    batch['text'] = [text.replace('\n', ' ') for text in batch['text']]
    return batch

# apply preprocessing to the dataset
dataset = dataset.map(preprocess, batched=True)

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

In [4]:
def tokenize_function(examples):
    tokenized = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)
    tokenized['labels'] = tokenized['input_ids'].copy()  # set labels to be the same as input_ids
    return tokenized

tokenized_data = dataset.map(tokenize_function, batched=True)

In [5]:
import transformers
print(transformers.__version__)


4.51.3


In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",                      # Where to save model checkpoints
    eval_strategy="epoch",                       # Evaluate every epoch (renamed from evaluation_strategy)
    per_device_train_batch_size=4,               # Batch size during training
    per_device_eval_batch_size=4,                # Batch size during evaluation
    num_train_epochs=1,                          # Number of training epochs
    logging_dir='./logs',                        # Directory for logs (for TensorBoard)
    logging_steps=10,                            # Log every 10 steps
    save_total_limit=1,                          # Keep only 1 checkpoint
    save_strategy="epoch",                       # Save after each epoch (recommended to avoid saving too frequently)
    load_best_model_at_end=True,                 # Optional: Load best model for final evaluation
    report_to="tensorboard"                      # Enables logging to TensorBoard
)

In [7]:
train_data = tokenized_data.shuffle().select(range(int(0.8 * len(tokenized_data))))
eval_data = tokenized_data.shuffle().select(range(int(0.8 * len(tokenized_data)), len(tokenized_data)))

In [8]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data
)

In [10]:
trainer.train()

c:\Users\sony\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,3.709700,3.593934


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=50, training_loss=3.7392850494384766, metrics={'train_runtime': 266.6144, 'train_samples_per_second': 0.75, 'train_steps_per_second': 0.188, 'total_flos': 6532418764800.0, 'train_loss': 3.7392850494384766, 'epoch': 1.0})

In [11]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model\\tokenizer_config.json',
 './fine_tuned_model\\special_tokens_map.json',
 './fine_tuned_model\\vocab.json',
 './fine_tuned_model\\merges.txt',
 './fine_tuned_model\\added_tokens.json',
 './fine_tuned_model\\tokenizer.json')

In [13]:
prompt = "The script"
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)  # Enable padding and truncation

# If pad_token isn't set, use eos_token (common for GPT models)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Generate with proper attention_mask and pad_token_id
output = model.generate(
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask'],  # Pass attention mask
    max_length=15,
    pad_token_id=tokenizer.pad_token_id  # Explicitly set pad token
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

The script is a bit of a mess, but it's worth the effort
